In [1]:
!pip install moviepy

In [2]:
import cv2
import numpy as np
import AddWatermark
from time import time
from evaluate2 import KModel
from threading import Barrier, Thread
from time import ctime, sleep

# BAR = Barrier(6)
styles_list = ['rain_princess', 'la_muse', 'scream', 'udnie', 'wave', 'wreck']
# styles_list = ['rain_princess']
rain_princess_model = KModel('pb_pretrian/pb/rain_princess.pb')
la_muse_model = KModel('pb_pretrian/pb/la_muse.pb')
scream_model = KModel('pb_pretrian/pb/scream.pb')
udnie_model = KModel('pb_pretrian/pb/udnie.pb')
wave_model = KModel('pb_pretrian/pb/wave.pb')
wreck_model = KModel('pb_pretrian/pb/wreck.pb')

MODELS_DICT ={
    'rain_princess': rain_princess_model, 
    'la_muse': la_muse_model, 
    'scream': scream_model, 
    'udnie': udnie_model, 
    'wave': wave_model, 
    'wreck':wreck_model
}

class Inf_Style():
    def __init__(self, name):
        self.name = name
    
    def start(self):
        self.thread = Thread(target=self.infrence_style, args=())
        self.thread.start()
        return self
        
    def infrence_style(self):
        print(self.name)
        MODELS_DICT[self.name].test(in_path='folder1', out_path=f'folder2/{self.name}/' )
        self.img = cv2.imread(f'folder2/{self.name}/photo0.png')
#         BAR.wait()
#         print('Something')
#         cv2.imshow(f'style photo {i}', ins_list[i].img) 
        

ins_list = []
for ns in styles_list:
    ins_list.append(Inf_Style(ns))
        
cam = cv2.VideoCapture(0) #cv2.VideoCapture(1)
cv2.namedWindow("test")
vgu = cv2.imread('vgu.png')
#img_counter = 0
def concat_vh(list_2d): 
    return cv2.vconcat([cv2.hconcat(list_h) for list_h in list_2d])


print('Go To While Loop')
while True:

    ret, frame = cam.read()
    #frame = AddWatermark.AddWatermark(frame, vgu, (100,100), (0,0))
    #print(frame.shape) #480, 640, 3
    #frame = cv2.resize(frame, (480, 640)) 
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)

    k = cv2.waitKey(1)
    if k == ord(' '):
        print("Escape hit, closing...")
        break
    elif k == ord('c'):
        #img_name = "folder1/photo{}.png".format(img_counter)
        img_name = "folder1/photo0.png"
        cv2.imwrite(img_name, cv2.resize(frame, (640,480)))
        #print("{} written!".format(img_name))
        print("Capture!!!")
        #img_counter += 1
        
        tick = time()
        for i in range(len(styles_list)):
            ins_list[i].start()
            
        for i in range(len(styles_list)):
            ins_list[i].thread.join()
        print(time() -tick)
#         row1 = [ins_list[i].img for i in range(0,3)]
#         row2 = [ins_list[i].img for i in range(3,6)]
        row1 = []
        row2 = []
        for i in range(len(styles_list)):
            if i < 3:
                row1.append(ins_list[i].img)
            else:
                row2.append(ins_list[i].img)
                
            cv2.imshow(f'style photo {i}', ins_list[i].img) 
        img_tile = concat_vh([row1, row2])
        cv2.imshow('final style photo ', img_tile)
        cv2.imwrite('folder2/res.png',img_tile)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

cam.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'cv2'